# Morals:

For Logistic Regression:

Optimal C depends on cv chosen (i.e. depends on random seed)

It might vary about 2 times e.g. from 0.0014 to  0.0024

However if determine C by one seed, and calculate score for that C in the other seed , the difference with the optimal seed for that score would be quite low - about 2*4-th digit of the logloss

CV score varies in 3-th digit e.g. from  -0.0881  to  -0.0871 depending in random seed


Here it is demonstrated on one target: "cyclooxygenase_inhibitor" (435 times contains 1, so class disbalance is less than for most other targets), but seems similar for many of them. Note that the target is not good predictable - constact prediction gives logloss: 0.0912, while models around 0.087-0.088. It might be for other targets situation is slighly less problematic. 


changing C 1 percent changes score in 7-th digit


Further results in: 
https://www.kaggle.com/alexandervc/moa30b-cyclooxygenase-inhibitor-c-interv
In particular seeds used in the present notebook -are delibrately taken to show the big difference.
Analysis on other sees can be found in cited notebook. 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns

import time
df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col = 0)  
df0 = df.copy()
df['cp_type'] = df['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':1.0}) # Forget about control group  
df['cp_dose'] = df['cp_dose'].map({'D1':0.0, 'D2':1.0})
df['cp_time'] = df['cp_time'].map({24:0.0, 48: .5 , 72:1.0})
X = df.copy()
X_save = X.copy()
df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',index_col = 0)
df0_test = df_test.copy()
df_test['cp_type'] = df_test['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':0.0})
df_test['cp_dose'] = df_test['cp_dose'].map({'D1':0.0, 'D2':1.0})
df_test['cp_time'] = df_test['cp_time'].map({24:0.0, 48: .5 , 72:1.0})

y = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv',index_col = 0 )
y_save = y.copy()
print(y.iloc[:3,:2])
df

In [ ]:
y = y_save['dopamine_receptor_antagonist']
y = y_save['cyclooxygenase_inhibitor']

y.sum()

In [ ]:
# LogLoss by constand prediction:
-log_loss(y, np.ones_like(y)*y.mean() )

In [ ]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

verbose = 1
df_stat = pd.DataFrame()
X_holdout = None
y_holdout = None

t00 = time.time()

for rs in [0,1999]:#,1,5, 100, 200, 113, 997, 57 , 1999, 10000]:
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= rs )
    
    for C in 0.002*np.arange(0.3,3,0.1):
        model = LogisticRegression(C=C)
        score_cv = cross_val_score(model, X,y, cv = skf , scoring = 'neg_log_loss')
        print(np.round(C,10),np.mean(score_cv), score_cv, time.time() - t00, 'seconds passed' )
        df_stat.loc[C,'CV Score RS'+str(rs)] = np.mean( score_cv )
        
print(time.time() - t00, 'seconds passed' )    

plt.figure(figsize = (20,6) )
for c in df_stat.columns:
    plt.plot(df_stat[c],'*-' ) # df_stat['CV Score']
plt.grid()
plt.show()

In [ ]:
l = []
for f in df_stat.columns:
    a = df_stat[f].argmax()
    c = df_stat.index[a]
    l.append(c)
np.min(l), np.max(l) 

In [ ]:
# Show difference between worst and best loglosses 
np.round(1e3* (df_stat.describe().loc['max',:].max() - df_stat.describe().loc['max',:].min()) , 1)

In [ ]:
df_stat.describe().loc['max',:]

In [ ]:
df_stat.max()-df_stat[f]

In [ ]:
# Calculate how much score suffes if we take optimal C for one seed and use it for another.

# Conclusion: suffers quite low 1.8*4-th digit 

l = []
for f in df_stat.columns:
    a = df_stat[f].argmax()
    c = df_stat.index[a]
    print(f, a, c )

e1 = np.round( 1e4* (df_stat['CV Score RS1999'].max() -df_stat['CV Score RS1999'].iloc[4]) , 1)
e2 = np.round( 1e4* (df_stat['CV Score RS0'].max() -df_stat['CV Score RS0'].iloc[9]) , 1)
e1,e2